## Web Scraping The-Numbers Movie Budgets

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import sqlite3
import matplotlib.pyplot as plt

MAX_PAGE = 5701
BASE_URL = 'https://www.the-numbers.com/movie/budgets/all/'

In [3]:
page = '1'
response = requests.get(BASE_URL + page)
response.status_code

200

In [5]:
soup = BeautifulSoup(response.text)

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1343128-1');
  </script>
  <meta content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
  <!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="telephone=no" name="format-detection"/>
  <!-- for apple mobile -->
  <meta content="521546213" property="fb:admins"/>
  <meta content="initial-scale=1" name="viewport"/>
  <meta content="The budge

In [8]:
tables = soup.find_all('table')
len(tables)

1

In [10]:
table = tables[0]

In [11]:
len(table.find_all('tr'))

101

In [12]:
len(table.find_all('th'))

6

In [13]:
for th in table.find_all('th'):
    print(th.text)

 
Release Date
Movie
Production Budget
Domestic Gross
Worldwide Gross


In [14]:
headers = [th.text for th in table.find_all('th')]
print(headers)

['\xa0', 'Release Date', 'Movie', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']


In [15]:
headers[0] = 'index'
print(headers)

['index', 'Release Date', 'Movie', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']


In [22]:
rows = table.find_all('tr')
data = []
# pick a simple
row = rows[11]
# print(row)
for row in rows[1:]:
    data_row = []
    tds = row.find_all('td')
    # first element only has text
    data_row.append(tds[0].text)
    # second element inside of <a> tag
    data_row.append(tds[1].find('a').text)
    # let's also grab the link from the second element
    data_row.append(tds[1].find('a')['href'])
    # element 3
    data_row.append(tds[2].find('a').text)    
    # element 3 link
    data_row.append(tds[2].find('a')['href'])
    # fourth element only has text
    data_row.append(tds[3].text)
    # fifth element only has text
    data_row.append(tds[4].text)
    # sixth element only has text
    data_row.append(tds[5].text)
#     print(*data_row, sep='\n')
    data.append(data_row)
    
print(len(data))
print(*[len(row) for row in data], sep='\n')

100
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


In [23]:
def process_page(soup):
    """
    Takes in a page from requests,
    outputs a list of lists (table format)
    """
    table = soup.find('table')
    rows = table.find_all('tr')
    data = []

    for row in rows[1:]:
        data_row = []
        tds = row.find_all('td')
        # first element only has text
        data_row.append(tds[0].text)
        # second element inside of <a> tag
        data_row.append(tds[1].find('a').text)
        # let's also grab the link from the second element
        data_row.append(tds[1].find('a')['href'])
        # element 3
        data_row.append(tds[2].find('a').text)    
        # element 3 link
        data_row.append(tds[2].find('a')['href'])
        # fourth element only has text
        data_row.append(tds[3].text)
        # fifth element only has text
        data_row.append(tds[4].text)
        # sixth element only has text
        data_row.append(tds[5].text)
        data.append(data_row)
    return data

In [34]:
from os.path import join
print(join(BASE_URL, '3'))

https://www.the-numbers.com/movie/budgets/all/3


In [35]:
from os.path import join
the_movies_data = []
for i in range(1, MAX_PAGE+1, 100):
    resp = requests.get(join(BASE_URL, str(i)))
    soup = BeautifulSoup(resp.text, 'html.parser')
    the_movies_data += process_page(soup)
    # same thing
    # the_movies_data.extend(process_page(soup))

print(len(the_movies_data))

5785


In [37]:
headers.insert(2, 'Release Date URL')
headers.insert(4, 'Movie URL')
print(headers)

['index', 'Release Date', 'Release Date URL', 'Movie', 'Movie URL', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']


In [46]:
headers[0] = 'i'
print(headers)

['i', 'Release Date', 'Release Date URL', 'Movie', 'Movie URL', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']


In [38]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [47]:
column_names = ','.join([header.replace(' ', '') + ' varchar\n' for header in headers])
print(column_names)

i varchar
,ReleaseDate varchar
,ReleaseDateURL varchar
,Movie varchar
,MovieURL varchar
,ProductionBudget varchar
,DomesticGross varchar
,WorldwideGross varchar



In [48]:
sql = """
CREATE TABLE movies ({col_names})
"""
cur.execute(sql.format(col_names=column_names))

In [51]:
cur.execute("select * from movies;")
cur.fetchall()

[('1',
  'Dec 18, 2009',
  '/box-office-chart/daily/2009/12/18',
  'Avatar',
  '/movie/Avatar#tab=summary',
  '$425,000,000',
  '$760,507,625',
  '$2,776,345,279')]

In [52]:
insert_sql = """
INSERT INTO movies VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""
# we've already insertedthe first row
for d in the_movies_data[1:]:
    cur.execute(insert_sql, d)

In [55]:
cur.execute("select Movie from movies limit 10 offset 5700;")
cur.fetchall()

[('The Night Visitor',),
 ('Tiger Orange',),
 ('The Last House on the Left',),
 ('The Foot Fist Way',),
 ('Dawn of the Crescent Moon',),
 ('Queen Crab',),
 ('Happy Christmas',),
 ('Peace, Propaganda and the Promised Land',),
 ('Absentia',),
 ('Pi',)]

In [57]:
pd.read_html('https://www.the-numbers.com/movie/budgets/all/401')

HTTPError: HTTP Error 403: Forbidden

In [59]:
resp.request.headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [65]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
user_agent = 'bogus'

In [66]:
headers = {'User-Agent': user_agent}
resp = requests.get('https://www.the-numbers.com/movie/budgets/all/401', headers=headers)

In [68]:
pd.read_html(resp.text)[0]

,Unnamed: 0,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,1,"Dec 25, 2015",Point Break,"$100,000,000","$28,782,481","$126,704,591"
1,2,"Dec 11, 2015",In the Heart of the Sea,"$100,000,000","$25,020,758","$89,693,309"
2,3,"Apr 18, 2014",Transcendence,"$100,000,000","$23,022,309","$103,039,258"
3,4,"Feb 18, 2005",Son of the Mask,"$100,000,000","$17,018,422","$59,918,422"
4,5,"Dec 14, 2018",Mortal Engines,"$100,000,000","$15,951,040","$85,287,417"
5,6,"Aug 16, 2002",The Adventures of Pluto Nash,"$100,000,000","$4,411,102","$7,094,995"
6,7,"Jan 20, 2012",Jin lÃ­ng shÃ­ san chai,"$100,000,000","$311,434","$98,227,017"
7,8,"Nov 6, 2015",The Peanuts Movie,"$99,000,000","$130,178,411","$250,091,610"
8,9,"Feb 8, 2019",The LEGO Movie 2: The Second Part,"$99,000,000","$105,806,508","$190,209,817"
9,10,"Nov 21, 2018",Robin Hood,"$99,000,000","$30,824,628","$84,747,441"
